https://scipython.com/blog/visualizing-a-vector-field-with-matplotlib/

In [10]:
import numpy as np
import k3d

In [11]:
def E(q, r0, x, y, z):
    """Return the electric field vector E=(Ex,Ey,Ez) due to charge q at r0."""
    den = np.hypot(x-r0[0], y-r0[1], z-r0[2])**3
    return q * (x - r0[0]) / den, q * (y - r0[1]) / den, q * (z - r0[2]) / den

# Grid of x, y points
nx, ny, nz = 20, 20, 1
x = np.linspace(-2, 2, nx, dtype=np.float32)
y = np.linspace(-2, 2, ny, dtype=np.float32)
z = np.linspace(-0, 2, nz, dtype=np.float32)
X, Y, Z = np.meshgrid(x, y, z)

charges = [(-1, (1.0, 0.0, 0)), (1, (-1.0, 0.0, 0))]
    
# Electric field vector, E=(Ex, Ey, Ez), as separate components
Ex, Ey, Ez = np.zeros((ny, nx, nz)), np.zeros((ny, nx, nz)), np.zeros((ny, nx, nz))
for charge in charges:
    ex, ey, ez = E(*charge, x=X, y=Y, z=Z)
    Ex += ex
    Ey += ey
    Ez += ez

Ex, Ey, Ez = np.ravel(Ex), np.ravel(Ey), np.ravel(Ez)
efield = np.stack([Ex, Ey, Ez]).T
efield /= np.stack([np.linalg.norm(efield, axis=1)]).T * 6
efield = np.float32(efield)

X, Y, Z = np.ravel(X), np.ravel(Y), np.ravel(Z)  

In [12]:
positive_charge = k3d.points([charges[1][1]], color=0xff0000 , point_size=0.1)
negative_charge = k3d.points([charges[0][1]], color=0x0000ff , point_size=0.1)
electric_field  = k3d.vectors(np.stack([X, Y, Z]).T, efield, head_size=0.5)

In [13]:
negative_charge + positive_charge + electric_field

Output()

In [14]:
ntc = 70

test_charges = []
for i in range(ntc):
    test_charges.append([np.cos(2*np.pi*i/ntc), np.sin(2*np.pi*i/ntc),0])
test_charges = np.array(test_charges)

#test_charges = np.random.uniform(-1,1, size=(ntc,3))  
#test_charges[:,2] *= 0

test_charges /= np.stack([np.linalg.norm(test_charges, axis=1)]).T * 10
test_charges = np.float32(test_charges)

plot = k3d.plot(camera_auto_fit=False, grid_auto_fit=False)
pcharges = k3d.points(test_charges+positive_charge.positions, color=0x00ff00, point_size=0.05)
ncharges = k3d.points(-1*test_charges+negative_charge.positions, color=0x00ffff, point_size=0.05)

plot += negative_charge + positive_charge + electric_field #+ pcharges + ncharges
plot.display()

Output()

In [15]:
pX, pY, pZ = pcharges.positions[:,0], pcharges.positions[:,1], pcharges.positions[:,2]
nX, nY, nZ = ncharges.positions[:,0], ncharges.positions[:,1], ncharges.positions[:,2]

ptrajectories, ntrajectories = [], []
for i in range(ntc):
    ptrajectories.append([])
    ntrajectories.append([])

for t in range(600):
    pEx, pEy, pEz = np.zeros(ntc), np.zeros(ntc), np.zeros(ntc)
    nEx, nEy, nEz = np.zeros(ntc), np.zeros(ntc), np.zeros(ntc)
    for charge in charges:
        pex, pey, pez = E(*charge, x=pX, y=pY, z=pZ)
        nex, ney, nez = E(*charge, x=nX, y=nY, z=nZ)
        pEx += pex
        pEy += pey
        pEz += pez
        nEx -= nex
        nEy -= ney
        nEz -= nez
        
    dt = np.float32(t/10000)
    pE = np.float32(np.vstack([pEx, pEy, pEz]).T)
    nE = np.float32(np.vstack([nEx, nEy, nEz]).T)
    
    pcharges.positions = pcharges.positions + pE*dt
    pX, pY, pZ = pcharges.positions[:,0], pcharges.positions[:,1], pcharges.positions[:,2]
    ncharges.positions = ncharges.positions + nE*dt
    nX, nY, nZ = ncharges.positions[:,0], ncharges.positions[:,1], ncharges.positions[:,2]
    
    p = np.vstack([pX,pY,pZ]).T
    n = np.vstack([nX,nY,nZ]).T
    for i in range(ntc):
        ptrajectories[i].append(p[i])
        ntrajectories[i].append(n[i])

In [16]:
ptrajectories = np.array(ptrajectories)
ptrajectories = list(ptrajectories)

ntrajectories = np.array(ntrajectories)
ntrajectories = list(ntrajectories)

In [17]:
for i in range(ntc):
    cond1 = (np.sum(ptrajectories[i] < np.array([2.0, 2.0, 2.0]), axis=1)//3).astype(np.bool)
    cond2 = (np.sum(ptrajectories[i] > np.array([-2.0, -2.0, -2.0]), axis=1)//3).astype(np.bool)
    square_cond = cond1 & cond2
    try:
        border = np.where(square_cond == False)[0][0]
    except IndexError:
        border = None
    ptrajectories[i] = ptrajectories[i][:border]
    
    neg_area_cond = np.sqrt(np.sum((ptrajectories[i] - negative_charge.positions)**2, axis=1)) > 0.25
    try:
        border = np.where(neg_area_cond == False)[0][0]
    except IndexError:
        border = None

    ptrajectories[i] = ptrajectories[i][:border]
    
    cond1 = (np.sum(ntrajectories[i] < np.array([2.0, 2.0, 2.0]), axis=1)//3).astype(np.bool)
    cond2 = (np.sum(ntrajectories[i] > np.array([-2.0, -2.0, -2.0]), axis=1)//3).astype(np.bool)
    square_cond = cond1 & cond2
    try:
        border = np.where(square_cond == False)[0][0]
    except IndexError:
        border = None
    ntrajectories[i] = ntrajectories[i][:border]
    
    pos_area_cond = np.sqrt(np.sum((ntrajectories[i] - positive_charge.positions)**2, axis=1)) > 0.25
    try:
        border = np.where(pos_area_cond == False)[0][0]
    except IndexError:
        border = None

    ntrajectories[i] = ntrajectories[i][:border]

In [18]:
for i in range(ntc):
    plot += k3d.line(ptrajectories[i], shader='simple', color=0x000000)
    plot += k3d.line(ntrajectories[i], shader='simple', color=0x00000)